## Following Along Youtube Tutorial

In [2]:
# import packages 

import random
import gym
import numpy as np
import keras 
import tensorflow 

In [3]:
# Create our enviornment and also the actions and states.

env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

# We now construct the env

episodes = 10 

for episode in range(1, episodes + 1):
  state = env.reset()
  done = False 
  score = 0

  while not done:
    env.render()
    action = random.choice([0,1])
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score:{}'.format(episode,score)) 

In [4]:
# Deep Learning Part of the thing. 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam 

In [11]:
# Build a NN model. 

def buildmodel(states,actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model 

In [12]:
# Create an instance 

model = buildmodel(states,actions)

In [13]:
# view model

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Import again

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [15]:
# Build the RL agent 

def build_agent(model,actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    return dqn

In [16]:
# Train the DQN Model to learn the game.

dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=50000,visualize= False, verbose =1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 10:39 - reward: 1.0000

C:\Users\tradk\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\tradk\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 150s 15ms/step - reward: 1.00000s - reward: 
114 episodes - episode_reward: 87.140 [9.000, 200.000] - loss: 3.467 - mae: 19.391 - mean_q: 39.154

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 128s 13ms/step - reward: 1.0000
50 episodes - episode_reward: 199.640 [188.000, 200.000] - loss: 8.330 - mae: 41.130 - mean_q: 82.821

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 136s 14ms/step - reward: 1.0000
50 episodes - episode_reward: 198.240 [170.000, 200.000] - loss: 8.318 - mae: 43.375 - mean_q: 87.207

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 170s 17ms/step - reward: 1.0000
51 episodes - episode_reward: 196.608 [161.000, 200.000] - loss: 7.194 - mae: 40.689 - mean_q: 81.569

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 147s 15ms/step - reward: 1.0000
done, took 731.271 seconds


In [19]:
# Lets see the scores of our trained model.

scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.historytory['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

AttributeError: 'History' object has no attribute 'historytory'

In [21]:
# For future references, we will save the weights so that we don't need to retrain for a future model.

dqn.save_weights('dqn_weights.h5f',overwrite = True)

In [24]:
# Now delete everything 

del model
del dqn 
del env

In [25]:
# Verify that we did indeed

dqn.test(env, nb_episodes=100, visualize=False)

NameError: name 'dqn' is not defined

In [30]:
# Now we remake it and load the weights to see it did work indeed

env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = buildmodel(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-3), metrics = ['mae'])

In [31]:
dqn.load_weights('dqn_weights.h5f')

In [32]:
# We see that the weights have indeed been loaded once again. 

dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...


C:\Users\tradk\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
